<a href="https://colab.research.google.com/github/safoura-banihashemi/Augmentation_Legal_Texts/blob/main/Augmentation_NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from collections import Counter
from itertools import chain
import ast

## Exploring Dataset

In [ ]:
# Load dataset
data = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/demosthenes.pkl")

data.head(10)

,Document,Split,Name,Id,Text,Type,Scheme,Start
0,A2008_Commission of the European Communities v...,1,prem,A1,"It must be held, first, that while Article 4(c...",L,Rule,18211
1,A2008_Commission of the European Communities v...,1,prem,A2,"Also, it is clear from consistent case-law tha...",L,"[Itpr, Prec, Rule]",18738
2,A2008_Commission of the European Communities v...,1,prem,A3,The Court deduces from this that Article 67 CS...,L,"[Prec, Rule]",19176
3,A2008_Commission of the European Communities v...,1,prem,A4,The Court has also held that action taken unde...,L,"[Prec, Rule]",19558
4,A2008_Commission of the European Communities v...,1,prem,A5,"The Court has held, in particular, that it is ...",L,"[Itpr, Prec, Rule]",19849
5,A2008_Commission of the European Communities v...,1,prem,A6,The Court has also held that the first indent ...,L,"[Prec, Rule]",20364
6,A2008_Commission of the European Communities v...,1,prem,A7,The Court accordingly held that a preferential...,L,"[Prec, Rule]",20757
7,A2008_Commission of the European Communities v...,1,prem,A8,"Finally, State aid granted to an undertaking f...",L,Itpr,21051
8,A2008_Commission of the European Communities v...,1,prem,A9,"In the light of the foregoing, it must consequ...",L,"[Itpr, Rule]",21327
9,A2008_Commission of the European Communities v...,1,prem,A10,"Contrary to what the German Government claims,...","[F, L]","[Itpr, Rule]",21545


In [ ]:
data.sample(10)

,Document,Split,Name,Id,Text,Type,Scheme,Start
2352,"R2021_Prosegur Compañía de Seguridad SA, estab...",5,prem,D14,"In that regard, it must be borne in mind that,...",L,Itpr,64441
586,A2017_European Commission v Italian Republic_DT,4,prem,B15,"Thus, it follows from that decision that, in t...",F,NaN,27452
627,A2017_European Commission v TV2_Danmark A_S,5,prem,A12,"As stated by the General Court, in paragraph 1...",F,NaN,22222
731,A2018_Commission v Spain,5,prem,D23,As regards the shipping companies and the ship...,F,NaN,71933
795,A2018_Dirk Andres v European Commission,5,prem,C19bis,the consequence of this would be that national...,L,Itpr,49038
2149,R2017_Viasat Broadcasting UK Ltd v European Co...,3,prem,B2bis,"second, the intervention must be liable to aff...",L,Class,21263
2222,R2021_FVE Holýšov I and Others v Commission,5,prem,E3,"Secondly, since the review mechanism must be r...","[F, L]",NaN,36203
988,R2000_French Republic v Ladbroke Racing Ltd an...,1,conc,A35,The first part of the first ground of appeal m...,NaN,NaN,33203
851,A2018_Scuola Elementare Maria Montessori Srl v...,3,prem,B8,"It must nonetheless be recalled that, if the g...",L,Prec,26354
1654,R2015_European Commission v MOL Magyar Olaj- é...,3,prem,A1,"It must be observed at the outset that, as the...",L,"[Aut, Class, Rule]",30917


In [ ]:
print(data['Split'].value_counts())

Split
5    702
4    535
1    483
3    448
2    367
Name: count, dtype: int64


In [ ]:
def analyze_multilabel_column(data, column_name):
    # Filter non-null values
    labeled_data = data[data[column_name].notnull()].copy()

    # Parse strings into lists
    def parse(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except:
                return [val]
        return val

    labeled_data[column_name] = labeled_data[column_name].apply(parse)

    # Count each item occurrence
    all_items = list(chain.from_iterable(labeled_data[column_name]))
    item_counts = Counter(all_items)

    # Convert to DataFrame
    item_counts_df = pd.DataFrame(item_counts.items(), columns=[column_name, 'Count']).sort_values(by='Count', ascending=False)

    # Count number of labels per example
    labeled_data[f'Num_{column_name}s'] = labeled_data[column_name].apply(len)
    multi_label_distribution = labeled_data[f'Num_{column_name}s'].value_counts().sort_index()
    multi_label_distribution_df = multi_label_distribution.reset_index().rename(columns={'index': 'Num_Labels', f'Num_{column_name}s': 'Count'})

    # Show results
    print(f"=== {column_name} Distribution ===")
    print(item_counts_df)

    print(f"\n=== Multi-label Distribution for {column_name} ===")
    print(multi_label_distribution_df)


# print results
analyze_multilabel_column(data, 'Type')
print(40 * "=")
analyze_multilabel_column(data, 'Name')
print(40 * "=")
analyze_multilabel_column(data, 'Scheme')

=== Type Distribution ===
  Type  Count
1    F   1576
0    L    906

=== Multi-label Distribution for Type ===
   Count  count
0      1   2268
1      2    107
=== Name Distribution ===
   Name  Count
0  prem   2375
1  conc    160

=== Multi-label Distribution for Name ===
   Count  count
0      1   2535
=== Scheme Distribution ===
  Scheme  Count
2   Prec    504
0   Rule    323
1   Itpr    298
3  Class     56
5    Aut     53
4  Princ     16

=== Multi-label Distribution for Scheme ===
   Count  count
0      1    615
1      2    253
2      3     43


Explaination of Scheme labels

| **Scheme** | **Meaning**               | **Typical Use in Legal Reasoning**                                                                                                                     |
| ---------- | ------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **Rule**   | **Application of a rule** | A legal norm is stated or applied to facts. <br> *E.g., “According to Article 10, the act is prohibited.”*                                             |
| **Prec**   | **Precedent**             | Refers to prior case law or decisions. <br> *E.g., “In *Smith v. Jones*, the court held that...”*                                                      |
| **Itpr**   | **Interpretation**        | Interprets laws or legal terms. <br> *E.g., “The term ‘public’ in this context means...”*                                                              |
| **Class**  | **Classification**        | Classifies a case/fact under a legal category. <br> *E.g., “This conduct falls under unfair competition.”*                                             |
| **Aut**    | **Authority**             | Appeals to legal authority beyond precedent (e.g., legal doctrine, scholars, institutions). <br> *E.g., “According to the Commission’s guidelines...”* |
| **Princ**  | **Principle**             | Refers to abstract legal principles (like justice, equality). <br> *E.g., “This violates the principle of proportionality.”*                           |

615 sentences are annotated with exactly 1 scheme

253 sentences have 2 scheme labels

43 sentences are annotated with 3 different schemes

In [ ]:
import ast

def parse_column_to_list(data, column_name):
    def parse(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except:
                return [val]
        return val
    data[column_name] = data[column_name].apply(parse)
    return data

def get_multi_label_counts(data, column_name, target_labels):
    # Ensure the column is in list format
    data = parse_column_to_list(data, column_name)

    # Filter data for each target label
    inclusive_data = {
        label: data[data[column_name].apply(lambda x: isinstance(x, list) and label in x)].copy()
        for label in target_labels
    }

    # Count entries per label
    label_counts = {label: len(df_) for label, df_ in inclusive_data.items()}
    return label_counts

# Example usage:
scheme_counts = get_multi_label_counts(data, 'Scheme', ['Class', 'Aut', 'Princ'])
name_counts = get_multi_label_counts(data, 'Name', ['conc'])

print("Scheme counts:", scheme_counts)
print("Name counts:", name_counts)

Scheme counts: {'Class': 56, 'Aut': 53, 'Princ': 16}
Name counts: {'conc': 160}


## Augmentation

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2025-05-31 14:26:27--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-05-31 14:26:28--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.11MB/s    in 2m 41s  

2025-05-31 14:29:10 (5.09 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [ ]:
!pip install nltk

In [ ]:
import nltk

# Download all required resources
nltk.download('punkt_tab')                        # Tokenizer
nltk.download('averaged_perceptron_tagger_eng')   # POS tagger
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')                          # WordNet for synonyms
nltk.download('omw-1.4')                          # WordNet multilingual data (helps with lemma mapping)
nltk.download('stopwords')                        # Stop Words

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords,wordnet
import string
import numpy as np
import re
import ast
import random
from nltk import word_tokenize, pos_tag
import nltk
from nltk.corpus import wordnet

In [ ]:
# Testing
sentence = "The judge delivered the final decision."
tokens = word_tokenize(sentence)
tags = pos_tag(tokens)

print(tags)

[('The', 'DT'), ('judge', 'NN'), ('delivered', 'VBD'), ('the', 'DT'), ('final', 'JJ'), ('decision', 'NN'), ('.', '.')]


In [ ]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'his', 'shan', "he'll", 'himself', 'ma', 'hadn', 'here', 'against', "i'd", 'nor', 'was', 'ourselves', 'are', 'herself', 'she', 'yourself', "you'd", 'which', 'into', 'o', 'our', 'most', 'hers', 'they', 'above', 'haven', 'my', 'not', 'hasn', 've', 'you', "she'll", 'can', 'as', 'each', 'when', 'it', 'how', 'me', 'a', 'am', 'this', 'and', 'be', 'by', "it'll", 'yours', 'after', 'these', 'under', 'any', "weren't", "shouldn't", 'yourselves', 'of', 'those', 'is', 'other', 'with', 'does', 'once', 'from', 'have', 'no', 'wouldn', 'or', 'why', 'did', "hasn't", 'only', "shan't", 'more', 'such', 'through', 'up', 'but', "you're", "don't", 'don', 'until', "hadn't", 'all', 'll', 'mightn', 'about', 'being', 'didn', 'down', 'ours', 'to', 'them', 'were', "you've", "doesn't", 'for', "aren't", 'been', "wasn't", 'theirs', 'during', 'should', "mightn't", 'too', "we'd", "it'd", 'own', "they'll", "you'll", 'just', "couldn't", 'so', "we've", "she's", 'itself', "they're", "mustn't", 'isn', "they've", 'has', 'sho

In [ ]:
# Path to GloVe file
glove_path = '/content/glove.6B.100d.txt'

# Initialize the embedding dictionary
embeddings = {}

# Read the file line by line
with open(glove_path, 'r', encoding='utf8') as file:
    for line in file:
        parts = line.strip().split()
        word = parts[0]
        vector = np.array(parts[1:], dtype='float32')
        embeddings[word] = vector

In [ ]:
print(len(embeddings))

400000


In [ ]:
def cosine_similarity(vec1, vec2):
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return -1
    return np.dot(vec1, vec2) / (norm1 * norm2)

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def most_similar_word(word, candidates, embeddings):
    """
    Given a word and a list of synonym candidates,
    return the one most similar to the word using GloVe cosine similarity.
    """
    if word not in embeddings:
        return candidates[0]  # fallback

    base_vec = embeddings[word].reshape(1, -1)  # make 2D for sklearn
    best_score = -1
    best_candidate = candidates[0]

    for candidate in candidates:
        candidate_words = candidate.lower().replace('_', ' ').replace('-', ' ').split()
        vectors = [embeddings[w] for w in candidate_words if w in embeddings]

        if not vectors:
            continue  # skip if none of the words have embeddings

        candidate_vec = np.mean(vectors, axis=0).reshape(1, -1)  # 2D for sklearn
        score = cosine_similarity(base_vec, candidate_vec)[0][0]

        if score > best_score:
            best_score = score
            best_candidate = candidate

    return best_candidate.replace('_', ' ')

def synonyms(word, n_desired=5):
    """
    Find up to n_desired synonyms for the input word using WordNet,
    prioritizing those with the same POS tag.
    """
    synsets = wordnet.synsets(word)
    if not synsets:
        return [], 0

    # Get POS tag of input word (e.g., 'N', 'J', 'R', etc.)
    word_pos = pos_tag([word])[0][1][0]

    candidates = []
    priority_flags = []

    for syn in synsets:
        for lemma in syn.lemmas():
            lemma_name = lemma.name().lower()

            # Skip the original word and duplicates
            if lemma_name == word.lower() or lemma_name in candidates:
                continue

            # Prioritize matching POS types
            syn_pos = syn.pos()
            if (word_pos == 'N' and syn_pos == 'n') or \
               (word_pos == 'J' and syn_pos in ['a', 's']) or \
               (word_pos == 'R' and syn_pos == 'r'):
                priority = 1
            else:
                priority = 0

            candidates.append(lemma_name)
            priority_flags.append(priority)

    if not candidates:
        return [], 0

    # Reorder candidates: matching POS tags first
    prioritized = [w for w, p in zip(candidates, priority_flags) if p == 1]
    non_prioritized = [w for w, p in zip(candidates, priority_flags) if p == 0]
    result = prioritized + non_prioritized

    return result[:n_desired], 1

In [ ]:
# Example
word = "principle"
candidates = ["rule", "doctrine", "precept", "code_of_ethics"]

best = most_similar_word(word, candidates, embeddings)
print("Best synonym:", best)

Best synonym: doctrine


In [ ]:
# Example
words, found = synonyms("principle", 5)
print(words)

['rule', 'precept', 'rationale']


In [ ]:
all_words = []
for line in data['Text']:
    tokens = nltk.word_tokenize(line.lower())
    all_words.extend(tokens)

print(Counter(all_words).most_common(30))  # See most common words

[('the', 14050), (',', 10213), ('of', 7829), ('in', 4437), ('that', 4355), ('to', 4137), ('and', 2719), ('it', 1727), ('a', 1656), ('by', 1490), ('not', 1437), ('is', 1431), ('court', 1405), ('commission', 1370), ('as', 1361), ('be', 1267), ('general', 1107), ('paragraph', 1070), ('which', 1004), ('judgment', 984), ('(', 963), (')', 963), ('on', 960), ('appeal', 936), ('aid', 913), ('from', 828), ('under', 818), ('for', 783), ('at', 772), ('was', 671)]


## Cleaning Dataset

In [ ]:
# Define the pattern for unusual characters (non-alphanumeric, non-standard punctuation)
pattern = r'[^a-zA-Z0-9\s\.,;!?\'\"()\-]'

# Extract just the unusual characters per line
def extract_unusual(text):
    return re.findall(pattern, text)

# Apply to your dataset
unusual_chars = data['Text'].apply(extract_unusual)

# Filter rows that have any unusual characters
unusual_rows = data[unusual_chars.apply(lambda x: len(x) > 0)]

# Show index + unusual characters for sample rows
for idx, chars in unusual_chars[unusual_rows.index].sample(30, random_state=1).items():
    print(f"Index {idx} → Unusual characters: {set(chars)}")

Index 1804 → Unusual characters: {':', '/', '‑'}
Index 856 → Unusual characters: {'’'}
Index 2394 → Unusual characters: {'’'}
Index 1578 → Unusual characters: {']', '[', '/'}
Index 1850 → Unusual characters: {'ü'}
Index 468 → Unusual characters: {':', '‑', '‘', '’', '/'}
Index 2029 → Unusual characters: {'é'}
Index 190 → Unusual characters: {']', '[', '/', '‑'}
Index 862 → Unusual characters: {'’'}
Index 120 → Unusual characters: {']', '[', '/', '‑'}
Index 1705 → Unusual characters: {':', '/', '‑'}
Index 112 → Unusual characters: {'’'}
Index 2063 → Unusual characters: {'’'}
Index 481 → Unusual characters: {'’', 'ñ'}
Index 1740 → Unusual characters: {'’'}
Index 2057 → Unusual characters: {'’'}
Index 327 → Unusual characters: {'’', '/'}
Index 1596 → Unusual characters: {'’', '–'}
Index 104 → Unusual characters: {'’'}
Index 1583 → Unusual characters: {']', '[', '‘'}
Index 420 → Unusual characters: {'’', '—'}
Index 367 → Unusual characters: {'š'}
Index 342 → Unusual characters: {']', '[', 

In [ ]:
# Define the pattern: anything NOT a-z, A-Z, 0-9, whitespace, or basic punctuation
pattern = r'[^a-zA-Z0-9\s\.,;!?\'\"()-]'

# Extract all unusual characters across the dataset
all_unusual = set()

for line in data['Text']:
    chars = re.findall(pattern, line)
    all_unusual.update(chars)

# Sort and print
unusual_sorted = sorted(all_unusual)
print("Unusual characters found:", unusual_sorted)
print(f"\nTotal unique unusual characters: {len(unusual_sorted)}")

Unusual characters found: ['%', '/', ':', '>', '[', ']', '`', 'Á', 'É', 'à', 'á', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'í', 'ï', 'ñ', 'ó', 'ö', 'ú', 'ü', 'ý', 'ě', 'ő', 'š', 'ű', 'ό', '‑', '–', '—', '‘', '’', '“', '”', '…']

Total unique unusual characters: 38


In [ ]:
def cleaning(text, track_unusual=False):
    """
    Cleans a line of text by:
    - Removing HTML/XML tags
    - Lowercasing text
    - Removing numbers
    - Removing punctuation and known unusual characters
    - Normalizing whitespace
    Returns:
        cleaned_text if track_unusual is False
        (cleaned_text, set_of_unusual_characters) if track_unusual is True
    """
    original_text = text
    text = text.lower()

    # Remove tags like <ref>
    text = re.sub(r'<.*?>', '', text)

    # Remove numbers
    text = re.sub(r'\d+', ' ', text)

    # Remove known unusual characters
    for ch in unusual_sorted:
        text = text.replace(ch, ' ')

    # Remove remaining punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    if track_unusual:
        # Extract characters not in normal ranges (for inspection)
        unusual = set(re.findall(r'[^a-zA-Z0-9\s\.,;!?\'\"()-]', original_text))
        return text, unusual
    else:
        return text

In [ ]:
# Example
sample = "This is a <tag>sample</tag> line… with ‘quotes’ and 2024 numbers—also punctuation!"
print(cleaning(sample))

this is a sample line with quotes and numbers also punctuation


## Augmentation (WordNet+GloVe)

In [ ]:
def augWN_GV(text, embeddings, stop_words, percent=0.6):
    """
    Apply WordNet+GloVe augmentation:
    - Replace a % of non-stopword nouns/adjectives/adverbs with WordNet synonyms,
      selected by highest cosine similarity using GloVe vectors.
    """
    # Tokenize, clean, lowercase
    tokens = nltk.word_tokenize(cleaning(text).capitalize())
    tokens = [t.lower() for t in tokens]

    # Remove stop words
    non_stop_tokens = [t for t in tokens if t not in stop_words]

    # POS tagging (on non-stopwords)
    tagged = nltk.pos_tag(non_stop_tokens)

    # Select replaceable tokens: adjectives, nouns, adverbs
    to_change = [t for t in tagged if t[1][0] in {'J', 'N', 'R'}]

    if not to_change:
        return text  # no eligible words

    # Determine how many words to replace
    n_replace = max(1, round(percent * len(to_change)))

    # For replacement tracking
    modified_text = text.lower()
    capitalized = text[0].isupper()
    changed = 0
    index = -1

    # Try replacements until desired count is met
    while changed < n_replace:
        index += 1
        if index >= len(to_change):
            break  # no more candidates

        word = to_change[index][0]

        # Get synonyms
        candidates, found = synonyms(word, 10)
        if not found or not candidates:
            continue

        # Pick the best synonym based on GloVe
        replacement = most_similar_word(word, candidates, embeddings)
        if replacement == word:
            continue  # skip if no real change

        # Replace only one occurrence (surrounded by non-alphabetic)
        pattern = rf'(?<![a-zA-Z#]){re.escape(word)}(?![a-zA-Z#])'
        modified_text = re.sub(pattern, f'#{replacement}#', modified_text, 1)
        changed += 1

    # Clean up: remove # markers and restore capitalization
    final = modified_text.replace('#', '')
    if capitalized:
        final = final.capitalize()
    return final

## Augmentation (WordNet + POS-based)

In [ ]:
def synonyms_pos_based(word, pos_tag):
    """Get synonyms from WordNet for a given word and simplified POS."""
    pos_map = {'J': wordnet.ADJ, 'N': wordnet.NOUN, 'R': wordnet.ADV}
    wn_pos = pos_map.get(pos_tag[0])
    if not wn_pos:
        return []

    synsets = wordnet.synsets(word, pos=wn_pos)
    lemmas = set()
    for syn in synsets:
        for lemma in syn.lemmas():
            name = lemma.name().replace('_', ' ').lower()
            if name != word.lower():
                lemmas.add(name)
    return list(lemmas)

def augWN_POS(text, stop_words, percent=0.6):
    """
    Apply WordNet+POS augmentation:
    - Replace a % of non-stopword adjectives/nouns/adverbs with random WordNet synonyms
    (based on POS, no embeddings used).
    """
    tokens = nltk.word_tokenize(cleaning(text).capitalize())
    tokens = [t.lower() for t in tokens]

    non_stop_tokens = [t for t in tokens if t not in stop_words]
    tagged = nltk.pos_tag(non_stop_tokens)

    to_change = [t for t in tagged if t[1][0] in {'J', 'N', 'R'}]
    if not to_change:
        return text

    n_replace = max(1, round(percent * len(to_change)))
    modified_text = text.lower()
    capitalized = text[0].isupper()
    changed = 0
    index = -1

    while changed < n_replace:
        index += 1
        if index >= len(to_change):
            break

        word, tag = to_change[index]
        candidates = synonyms_pos_based(word, tag)

        if not candidates:
            continue

        replacement = random.choice(candidates)
        if replacement == word:
            continue

        pattern = rf'(?<![a-zA-Z#]){re.escape(word)}(?![a-zA-Z#])'
        modified_text = re.sub(pattern, f'#{replacement}#', modified_text, 1)
        changed += 1

    final = modified_text.replace('#', '')
    if capitalized:
        final = final.capitalize()
    return final

## Renew the Dataset

In [ ]:
data_new = data.copy()
data_new['Augmentation_Flag'] = 'original'  # Mark originals

# Parse 'Scheme' and 'Name' columns to lists
def parse_list(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except:
            return [val]
    return val

data_new['Scheme'] = data_new['Scheme'].apply(parse_list)
data_new['Name'] = data_new['Name'].apply(parse_list)

# Define targets
target_schemes = ['Princ', 'Class', 'Aut']
target_names = ['conc']

# Filter and clean data
def prepare_filtered_data(column, targets):
    return data_new[data_new[column].apply(lambda x: isinstance(x, list) and any(t in x for t in targets))].copy()

filtered_scheme = prepare_filtered_data('Scheme', target_schemes)
filtered_name = prepare_filtered_data('Name', target_names)

# Clean text (assume you have a `cleaning` function)
filtered_scheme['Clean_Text'] = filtered_scheme['Text'].apply(cleaning)
filtered_name['Clean_Text'] = filtered_name['Text'].apply(cleaning)

# Augment text
filtered_scheme['Augmented_Text'] = filtered_scheme['Clean_Text'].apply(lambda x: augWN_GV(x, embeddings, stop_words))
filtered_name['Augmented_Text'] = filtered_name['Clean_Text'].apply(lambda x: augWN_GV(x, embeddings, stop_words))

# Build augmented rows keeping all original columns
def build_augmented_rows(df):
    aug_rows = df.copy()
    aug_rows['Text'] = aug_rows['Augmented_Text']  # Replace text with augmented version
    aug_rows['Augmentation_Flag'] = 'augmented'
    return aug_rows

augmented_scheme_df = build_augmented_rows(filtered_scheme)
augmented_name_df = build_augmented_rows(filtered_name)

# Concatenate everything
final_data = pd.concat([data_new, augmented_scheme_df, augmented_name_df], ignore_index=True)

# Drop temporary columns
final_data.drop(columns=['Clean_Text', 'Augmented_Text'], inplace=True, errors='ignore')

# Save
final_data.to_pickle("/content/drive/MyDrive/Colab Notebooks/final_augmented_data.pkl")
print(final_data[['Text', 'Augmentation_Flag']].sample(10))

                                                   Text Augmentation_Flag
77    It cannot be deduced from the fact that an agr...          original
78    It is difficult to see how the social policy o...          original
854   In so far as such entities were a priori eligi...          original
1125  Accordingly, the Court of First Instance did n...          original
2562  one-fourth it must deform or threaten to disto...         augmented
1266  In reality, it referred to the economic-policy...          original
1954  The appellant argued, in addition, that the am...          original
2381  In that regard, it is true that, in the decisi...          original
2660  therefore the one-third arm of the for the fir...         augmented
1051  Against that background, the Commission's clai...          original


In [ ]:
final_data = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/final_augmented_data.pkl")
final_data.sample(20)

,Document,Split,Name,Id,Text,Type,Scheme,Start,Augmentation_Flag
1835,R2016_European Commission v Hansestadt Lübeck,4,[prem],C11,"Accordingly, whilst it cannot be ruled out tha...",L,[Itpr],29883,original
2113,R2017_European Commission v Frucona Košice a,2,[prem],D3,"In that regard, all information liable to have...",L,[Prec],37893,original
186,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,[prem],B10bis,"Article 13(1) of Regulation No 659/1999, which...",L,[Rule],26119,original
211,A2011_European Commission (C-106_09 P) and Kin...,2,[prem],A3,It is necessary to reject from the outset the ...,F,NaN,40922,original
1804,R2016_European Commission v Hansestadt Lübeck,4,[prem],A10,"Indeed, it has neither contended nor establish...","[F, L]",[Prec],13096,original
2621,R2016_Orange v European Commission,4,[prem],A8,it should be noted as observe by the proponent...,F,[Aut],21322,augmented
2558,A2018_Commission v Spain,5,[prem],A10,according to the settled caselaw of the royal ...,L,"[Class, Prec, Rule]",24941,augmented
1139,R2004_Daewoo Electronics Manufacturing España ...,1,[prem],A2,"In particular, where aid is paid without prior...",L,"[Prec, Rule]",17325,original
1044,R2002_Associação dos Refinadores de Açúcar Por...,1,[prem],B9,It is clear from the documents on the Court fi...,F,NaN,39203,original
1811,R2016_European Commission v Hansestadt Lübeck,4,[prem],B6,It stated that such a decision necessarily alt...,F,NaN,17132,original


In [ ]:
print(final_data['Augmentation_Flag'].value_counts())

Augmentation_Flag
original     2535
augmented     283
Name: count, dtype: int64


In [ ]:
# Ensure both are copies to avoid warnings
filtered_data_scheme = filtered_scheme.copy()
filtered_data_name = filtered_name.copy()

# Augment Scheme data
filtered_data_scheme['Augmented_Text'] = filtered_data_scheme['Clean_Text'].apply(
    lambda x: augWN_GV(x, embeddings, stop_words)
)

with open('/content/augmented_scheme.txt', 'w', encoding='utf-8') as f:
    for orig, aug in zip(filtered_data_scheme['Clean_Text'], filtered_data_scheme['Augmented_Text']):
        f.write(f"ORIGINAL: {orig}\nAUGMENTED: {aug}\n\n")

# Augment Name data
filtered_data_name['Augmented_Text'] = filtered_data_name['Clean_Text'].apply(
    lambda x: augWN_GV(x, embeddings, stop_words)
)

with open('/content/augmented_name.txt', 'w', encoding='utf-8') as f:
    for orig, aug in zip(filtered_data_name['Clean_Text'], filtered_data_name['Augmented_Text']):
        f.write(f"ORIGINAL: {orig}\nAUGMENTED: {aug}\n\n")

print("Scheme augmented data saved to: /content/augmented_scheme.txt")
print("Name augmented data saved to: /content/augmented_name.txt")

Scheme augmented data saved to: /content/augmented_scheme.txt
Name augmented data saved to: /content/augmented_name.txt


In [ ]:
# print results
analyze_multilabel_column(final_data, 'Type')
print(40 * "=")
analyze_multilabel_column(final_data, 'Name')
print(40 * "=")
analyze_multilabel_column(final_data, 'Scheme')

=== Type Distribution ===
  Type  Count
1    F   1617
0    L    999

=== Multi-label Distribution for Type ===
   Count  count
0      1   2380
1      2    118
=== Name Distribution ===
   Name  Count
0  prem   2498
1  conc    320

=== Multi-label Distribution for Name ===
   Count  count
0      1   2818
=== Scheme Distribution ===
  Scheme  Count
2   Prec    538
0   Rule    349
1   Itpr    314
3  Class    112
5    Aut    106
4  Princ     32

=== Multi-label Distribution for Scheme ===
   Count  count
0      1    679
1      2    293
2      3     62


In [ ]:
# Step 1: Copy original data and mark
data_new2 = data.copy()
data_new2['Augmentation_Flag'] = 'original'

# Step 2: Parse 'Scheme' and 'Name' columns from strings to lists
def parse_list(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except:
            return [val]
    return val

data_new2['Scheme'] = data_new2['Scheme'].apply(parse_list)
data_new2['Name'] = data_new2['Name'].apply(parse_list)

# Step 3: Define target classes
target_schemes = ['Princ', 'Class', 'Aut']
target_names = ['conc']

# Step 4: Filter data by targets
def prepare_filtered_data(column, targets):
    return data_new2[data_new2[column].apply(lambda x: isinstance(x, list) and any(t in x for t in targets))].copy()

filtered_scheme = prepare_filtered_data('Scheme', target_schemes)
filtered_name = prepare_filtered_data('Name', target_names)

# Step 5: Clean text (assumes you have a cleaning(text) function)
filtered_scheme['Clean_Text'] = filtered_scheme['Text'].apply(cleaning)
filtered_name['Clean_Text'] = filtered_name['Text'].apply(cleaning)

# Step 6: Augment using POS-based WordNet augmentation (no embeddings used)
filtered_scheme['Augmented_Text'] = filtered_scheme['Clean_Text'].apply(lambda x: augWN_POS(x, stop_words))
filtered_name['Augmented_Text'] = filtered_name['Clean_Text'].apply(lambda x: augWN_POS(x, stop_words))

# Step 7: Build augmented versions while keeping all metadata
def build_augmented_rows(df):
    aug_rows = df.copy()
    aug_rows['Text'] = aug_rows['Augmented_Text']
    aug_rows['Augmentation_Flag'] = 'augmented'
    return aug_rows

augmented_scheme_df = build_augmented_rows(filtered_scheme)
augmented_name_df = build_augmented_rows(filtered_name)

# Step 8: Combine original + augmented
final_data_POS = pd.concat([data_new2, augmented_scheme_df, augmented_name_df], ignore_index=True)

# Step 9: Drop temporary columns
final_data_POS.drop(columns=['Clean_Text', 'Augmented_Text'], inplace=True, errors='ignore')

# Step 10: Save final dataset
final_data_POS.to_pickle("/content/drive/MyDrive/Colab Notebooks/final_augmented_data_POS.pkl")

# Optional: Sample output
print(final_data_POS[['Text', 'Augmentation_Flag']].sample(10))

                                                   Text Augmentation_Flag
601   First, it is apparent from paragraph 21 of the...          original
2657  in that gaze it is honest that in the decisive...         augmented
803   It also pointed out, in paragraph 104 of that ...          original
1942  In those circumstances, the second part of thi...          original
2026  It should be noted in that regard that, in par...          original
1532  As regards the question whether the applicabil...          original
2255  On the other hand, where it appears that such ...          original
2048  It follows that the first argument of the firs...          original
1646  Accordingly, the General Court was correct in ...          original
893   Consequently, contrary to Scuola Elementare Ma...          original


In [ ]:
# Ensure both are copies to avoid warnings
filtered_data_scheme = filtered_scheme.copy()
filtered_data_name = filtered_name.copy()

# Augment Scheme data using POS-based method
filtered_data_scheme['Augmented_Text'] = filtered_data_scheme['Clean_Text'].apply(
    lambda x: augWN_POS(x, stop_words)
)

with open('/content/augmented_scheme_POS.txt', 'w', encoding='utf-8') as f:
    for orig, aug in zip(filtered_data_scheme['Clean_Text'], filtered_data_scheme['Augmented_Text']):
        f.write(f"ORIGINAL: {orig}\nAUGMENTED: {aug}\n\n")

# Augment Name data using POS-based method
filtered_data_name['Augmented_Text'] = filtered_data_name['Clean_Text'].apply(
    lambda x: augWN_POS(x, stop_words)
)

with open('/content/augmented_name_POS.txt', 'w', encoding='utf-8') as f:
    for orig, aug in zip(filtered_data_name['Clean_Text'], filtered_data_name['Augmented_Text']):
        f.write(f"ORIGINAL: {orig}\nAUGMENTED: {aug}\n\n")

print("Scheme augmented data saved to: /content/augmented_scheme_POS.txt")
print("Name augmented data saved to: /content/augmented_name_POS.txt")

Scheme augmented data saved to: /content/augmented_scheme_POS.txt
Name augmented data saved to: /content/augmented_name_POS.txt


In [ ]:
# print results
analyze_multilabel_column(final_data, 'Type')
print(40 * "=")
analyze_multilabel_column(final_data, 'Name')
print(40 * "=")
analyze_multilabel_column(final_data, 'Scheme')

=== Type Distribution ===
  Type  Count
1    F   1617
0    L    999

=== Multi-label Distribution for Type ===
   Count  count
0      1   2380
1      2    118
=== Name Distribution ===
   Name  Count
0  prem   2498
1  conc    320

=== Multi-label Distribution for Name ===
   Count  count
0      1   2818
=== Scheme Distribution ===
  Scheme  Count
2   Prec    538
0   Rule    349
1   Itpr    314
3  Class    112
5    Aut    106
4  Princ     32

=== Multi-label Distribution for Scheme ===
   Count  count
0      1    679
1      2    293
2      3     62


## Evaluation

In [ ]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.1 MB/s eta 0:00:00


In [ ]:
import Levenshtein
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.util import ngrams

In [ ]:
# Lexical diversity functions
def jaccard_similarity(set1, set2):
    return len(set1 & set2) / len(set1 | set2) if set1 | set2 else 0

def evaluate_lexical_diversity(originals, augmentations):
    scores = {
        'BLEU': [],
        'Levenshtein': [],
        'Jaccard': [],
        'Token Change Ratio': []
    }

    for orig, aug in zip(originals, augmentations):
        tokens_orig = nltk.word_tokenize(orig)
        tokens_aug = nltk.word_tokenize(aug)

        bleu = sentence_bleu([tokens_orig], tokens_aug, smoothing_function=SmoothingFunction().method1)
        lev = Levenshtein.distance(orig, aug)
        jacc = jaccard_similarity(set(tokens_orig), set(tokens_aug))
        changed = sum(o != a for o, a in zip(tokens_orig, tokens_aug))
        ratio = changed / max(len(tokens_orig), 1)

        scores['BLEU'].append(bleu)
        scores['Levenshtein'].append(lev)
        scores['Jaccard'].append(jacc)
        scores['Token Change Ratio'].append(ratio)

    return {metric: sum(vals) / len(vals) for metric, vals in scores.items()}

In [ ]:
def load_augmented_pairs(file_path):
    """Read augmented .txt file and extract original/augmented sentence pairs."""
    originals, augmenteds = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    orig, aug = None, None
    for line in lines:
        if line.startswith("ORIGINAL:"):
            orig = line.replace("ORIGINAL:", "").strip()
        elif line.startswith("AUGMENTED:"):
            aug = line.replace("AUGMENTED:", "").strip()
            if orig is not None:
                originals.append(orig)
                augmenteds.append(aug)
                orig, aug = None, None  # reset pair
    return originals, augmenteds

In [ ]:
# GloVe-based
glove_scheme_orig, glove_scheme_aug = load_augmented_pairs("/content/augmented_scheme.txt")
glove_name_orig, glove_name_aug = load_augmented_pairs("/content/augmented_name.txt")

# POS-based
pos_scheme_orig, pos_scheme_aug = load_augmented_pairs("/content/augmented_scheme_POS.txt")
pos_name_orig, pos_name_aug = load_augmented_pairs("/content/augmented_name_POS.txt")

# Optionally, merge scheme + name for global comparison
glove_orig = glove_scheme_orig + glove_name_orig
glove_aug  = glove_scheme_aug + glove_name_aug

pos_orig = pos_scheme_orig + pos_name_orig
pos_aug  = pos_scheme_aug + pos_name_aug

In [ ]:
metrics_glove = evaluate_lexical_diversity(glove_orig, glove_aug)
metrics_pos   = evaluate_lexical_diversity(pos_orig, pos_aug)

print("Lexical Diversity Metrics - GloVe")
for k, v in metrics_glove.items():
    print(f"{k}: {v:.4f}")

print("\nLexical Diversity Metrics - POS")
for k, v in metrics_pos.items():
    print(f"{k}: {v:.4f}")

Lexical Diversity Metrics - GloVe
BLEU: 0.5206
Levenshtein: 52.4558
Jaccard: 0.6566
Token Change Ratio: 0.7935

Lexical Diversity Metrics - POS
BLEU: 0.5112
Levenshtein: 53.9576
Jaccard: 0.5909
Token Change Ratio: 0.5554
